In [1]:
import json
import pandas as pd

# Cargar los datos
with open('input/ttcw_annotations.json') as f:
    annotations = json.load(f)

with open('input/ttcw_short_stories.json') as f:
    short_stories = json.load(f)

with open('input/ttcw_all_tests.json') as f:
    tests = json.load(f)

# Convertir a DataFrames
df_annotations = pd.DataFrame(annotations)
df_short_stories = pd.DataFrame(short_stories)
df_tests = pd.DataFrame(tests)

# Calcular la puntuación total para cada historia por crítico
df_annotations['binary_score'] = df_annotations['binary_verdict'].apply(lambda x: 1 if x == "Yes" else 0)
df_scores = df_annotations.groupby(['expert_idx', 'story_id']).agg(total_score=('binary_score', 'sum')).reset_index()

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación total de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking: si hay empates se le asigna la misma posición (dense ranking)
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total
    group['ranking_value'] = group['total_score']
    # Calcular el ranking value normalizado por experto (min-max)
    min_val = group['ranking_value'].min()
    max_val = group['ranking_value'].max()
    if max_val - min_val != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_val) / (max_val - min_val)
    else:
        group['ranking_value_norm'] = 0.0
    return group

# Aplicar la función de ranking para cada experto
df_ranked = df_scores.groupby('expert_idx').apply(assign_ranking).reset_index(drop=True)
# Añadir la columna 'user_id' formateada como Expert{expert_idx}
df_ranked['user_id'] = 'Expert' + df_ranked['expert_idx'].astype(str)

# Seleccionar las columnas deseadas
df_output = df_ranked[['user_id', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']]

df_output.to_csv('output/ranking.csv', index=False)

/tmp/ipykernel_421110/3639219542.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df_scores.groupby('expert_idx').apply(assign_ranking).reset_index(drop=True)
